# Upsert AOOS Priority Score

## Approaching Out of Stock (AOOS)

* Priority scores of work items (inventories) in AOOS work queue are calculated and upserted to InfoHub
* The function `AOOS_priority_score` is defined below - for understanding the business logic, refer to the accompanying Notebook **AOOS-Priority-Score.ipynb**

## InfoHub

* The InfoHub connection and queries are defined in the accompanying Notebook **InfoHub.ipynb**
* Make sure that you have run the Kernel of the above Notebook

In [ ]:
import json
import sys
import os
import pandas as pd
import time
import numpy as np
import datetime
import json

### Import `InfoHubConnection` Class

* Install `ipynb` package for the following import to work

    `pip install ipynb`
    
* Make sure that the Kernel of Notebook **InfoHub.ipynb** has been run without errors

In [ ]:
from ipynb.fs.full.InfoHub import InfoHubConnection

### Priority Score Calculation

* Priority score function for _Approaching Out of Stock_ work item
* The business logic is explained in detail in the accompanying Notebook **AOOS-Priority-Score.ipynb**

In [ ]:
def AOOS_priority_score(supply_plans, 
                        demand_plans,
                        starting_inventory = 0,
                        first_date = datetime.date(2021, 7, 31), 
                        last_date = datetime.date(2021, 8, 31), 
                        decay_weight = 3.0, 
                        inv_positive_threshold = 20, 
                        inv_negative_threshold = -100):
    
    horizon = (last_date - first_date).days + 1
    # Define Inventory horizon and add starting inventory
    inventory_horizon = np.zeros(horizon, dtype=int)
    inventory_horizon = inventory_horizon + starting_inventory

    # Add Supply plans
    for i in range(len(supply_plans)):
        supply_date = datetime.datetime.fromisoformat(supply_plans[i]['startDate'][:-1]).date()
        qty = supply_plans[i]['quantity']
        diff_days = (supply_date - first_date).days
        inventory_horizon[diff_days:] = inventory_horizon[diff_days:] + qty

    for i in range(len(demand_plans)):
        demand_date = datetime.datetime.fromisoformat(demand_plans[i]['startDate'][:-1]).date()
        qty = demand_plans[i]['quantity']
        diff_days = (demand_date - first_date).days
        inventory_horizon[diff_days:] = inventory_horizon[diff_days:] - qty

    # Calculate weights
    weights = np.exp(np.arange(decay_weight, 0, -(decay_weight/horizon)))/np.exp(decay_weight)

    # Calculate penalty
    inventory_below_threshold = inventory_horizon - inv_positive_threshold
    penalties = np.zeros(horizon, dtype=int)
    neg_inv_mask = inventory_below_threshold < 0
    penalties[neg_inv_mask] = inventory_below_threshold[neg_inv_mask] 

    neg_threshold_mask = penalties < inv_negative_threshold
    penalties[neg_threshold_mask] = inv_negative_threshold

    total_penalty = np.sum(weights*-penalties)
    max_penalty = np.sum(weights*-inv_negative_threshold)
    priority_score = int(np.rint((total_penalty/max_penalty)*100))

    return priority_score


### InfoHub Connection Config

* Load your InfoHub connection parameters from `credentials.json` file
* `tenantId` is not required for establishing the connection, but is required in the GraphQL queries

In [ ]:
with open("credentials.json") as config_file:
    config = json.load(config_file)

In [ ]:
url = config['url']
headers  = config['headers']
tenantId = config['tenantId']
infohub = InfoHubConnection(url=url, tenantId=tenantId, headers=headers)

### Priority Score Config

* `timestamp`: Timestamp is needed in `upsert` operation, in ISO format.
    * In production, use the current system timestamp.
* `maxDate`: Active supply and demand plans till this date are used for priority score calculation
    * For more details, check the accompanying Notebook **AOOS-Priority-Score.ipynb**

In [ ]:
timestamp = "2021-08-03T10:37:07+0800"
maxDate = "2021-08-31 00:00:00"

### Work Queue List

* Our goal is to evaluate / re-calculate the priority score of work items in the AOOS work queue
* We need the AOOS work queue object ID to get the work items that are in progress
* To get the AOOS work queue object ID, we use the following query to get the list of work queues.

In [ ]:
workqueues = infohub.get_work_queues()
for i in range(len(workqueues)):
    print('WorkQueue: ', workqueues[i]['object']['name'])
    print('Id: ', workqueues[i]['object']['id'])
    print("--------------------------------------------------------------------------------")

### Choose `workQueueId`

* Choose the workQueueId of _Inventory approaching out of stock prioritized_

In [ ]:
workQueueId = "516dc12d-eff6-4c51-9222-7eca88a31c5e"

### Collect Work Items

* Given the work queue ID, collect all the work items

In [ ]:
work_items = []
print("Querying work items in progress...")
work_items_list = infohub.get_workitems_in_progress(workQueueId=workQueueId)
print("\tNumber of WorkItems: {}".format(len(work_items_list)))
for i in range(len(work_items_list)):
    wi = {"workItemId": work_items_list[i]['object']["id"],
          "priority": work_items_list[i]['object']["priority"],
          "inventoryId": work_items_list[i]['object']["businessObject"]["id"],
          "productId": work_items_list[i]['object']["businessObject"]["product"]["id"],
          "partNumber": work_items_list[i]['object']["businessObject"]["product"]["partNumber"],
          "locationId": work_items_list[i]['object']["businessObject"]["location"]["id"],
          "locationIdentifier": work_items_list[i]['object']["businessObject"]["location"]["locationIdentifier"],
          "starting_inventory": work_items_list[i]['object']["businessObject"]["quantity"]
    }
    work_items.append(wi)
    print("({}): Object-Id: {};".format(i, wi["workItemId"]))
    print("\tPart-Number: {}; Location-Id: {}; Priority-Score: {}".format(wi["partNumber"], wi["locationIdentifier"], wi["priority"]))
print('Done.')

### Upsert Priority Score

* Upsert the priority scores of each work item in 4 steps
    * Alternatively, choose only the work items of interest to calculate and upsert the score

**4 Steps**
* Steps 1 & 2: Get Supply and Demand Plans 
    * The priority score requires list of supply plans and demand plans of the inventory (of the respective work item)
    * We can query these plans using `inventoryId` or alternatively using `partNumber` and `locationIdentifier`
    * We use `partNumber` and `locationIdentifier` for readability and debugging
* Step 3: Calculate Priority Score
    * Uses `AOOS_priority_score` function
* Step 4: Upsert the new Priority Score
    * Uses the collected field `workItemId` and the user defined parameter `timestamp`
    * In production, predefined `timestamp` can be replaced with current time

In [ ]:
for k in range(len(work_items)):
    supply_plans = []
    demand_plans =[]
    partNumber = work_items[k]["partNumber"]
    locationIdentifier = work_items[k]["locationIdentifier"]
    starting_inventory = work_items[k]["starting_inventory"]
    workItemId = work_items[k]["workItemId"]

    print("PartNumber: {}; LocationIdentifier: {}; WorkItemID: {}".format(partNumber, locationIdentifier, workItemId))

    # Get supply plans
    print("1. Querying supply plans ...")
    plan_list = infohub.get_supplyplans(partNumber=partNumber, locationIdentifier=locationIdentifier, maxDate=maxDate)
    print("\tNumber of Supply Plans: {}".format(len(plan_list)))
    for i in range(len(plan_list)):
        plan = {"startDate": plan_list[i]['object']["startDate"],
                "quantity": plan_list[i]['object']["quantity"]}
        supply_plans.append(plan)

    # Get demand plans
    print("2. Querying demand plans ...")
    plan_list = infohub.get_demandplans(partNumber=partNumber, locationIdentifier=locationIdentifier, maxDate=maxDate)
    print("\tNumber of Demand Plans: {}".format(len(plan_list)))
    for i in range(len(plan_list)):
        plan = {"startDate": plan_list[i]['object']["startDate"],
                "quantity": plan_list[i]['object']["quantity"]}
        demand_plans.append(plan)

    print("3. Calculating priority score ...")
    priority_score = AOOS_priority_score(supply_plans=supply_plans, demand_plans=demand_plans, starting_inventory=starting_inventory, inv_positive_threshold=100, inv_negative_threshold=-300)
    print("\tPriority score: {}".format(priority_score))

    print("4. Upserting work item object ...")
    upsert_result = infohub.upsert_workitem_priority(workItemId=workItemId, priority=priority_score, timestamp=timestamp)
    print(upsert_result)
    print("----------")